In [1]:
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
import math

In [2]:
# file1 = sys.argv[1]
# file2 = sys.argv[2]
# result = sys.argv[3]

In [3]:
#This is found from:
#https://stackoverflow.com/questions/27928/calculate-distance-between-two-latitude-longitude-points-haversine-formula/21623206
def distance_TWO_points(lat1,lon1,lat2,lon2):
    R = 6371 # Radius of the earth in km
    dLat = math.radians(lat2-lat1)  #deg2rad below
    dLon = math.radians(lon2-lon1)
    a = np.sin(dLat/2) * np.sin(dLat/2) + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dLon/2) * np.sin(dLon/2)
    c = 2 * np.atan2(np.sqrt(a), np.sqrt(1-a))
    dis = R * c # Distance in km
    return dis
dis_F = np.vectorize(distance_TWO_points)

def distance(city, stations):
    stations['distance'] = dis_F(city['latitude'],city['longitude'],stations['latitude'],stations['longitude'])
    return stations['distance']

def best_tmax(city, stations):
    stations['distance'] = distance(city, stations)
    return stations['avg_tmax'][stations['distance'].idxmin()]

In [4]:
stations = pd.read_json('stations.json.gz', lines=True)
# stations = pd.read_json(file1, lines=True)

stations['avg_tmax'] = stations['avg_tmax']/10
# stations

In [5]:
city = pd.read_csv('city_data.csv').dropna()
# city = pd.read_csv(file2).dropna()

# print(city)
city['area'] = city['area']/1000000
city = city[city['area'] <= 10000].reset_index(drop=True)
city

,name,population,area,latitude,longitude
0,Calgary,1096833.0,825.290000,51.054444,-114.066944
1,Edmonton,812201.0,684.370000,53.500000,-113.500000
2,Abbotsford,133497.0,375.550000,49.054611,-122.328000
3,Burnaby,223218.0,90.610000,49.250000,-122.949167
4,Nanaimo,83811.0,91.000000,49.164167,-123.936389
...,...,...,...,...,...
334,Vista,93834.0,48.377000,33.193611,-117.241111
335,Renton,90927.0,60.970000,47.486667,-122.195278
336,San Angelo,93200.0,150.900000,31.450000,-100.450000
337,San Juan,355074.0,199.429084,18.450000,-66.066667


In [6]:
# The function blow is learned from https://blog.csdn.net/qq_19528953/article/details/79348929
city['best_tmax'] = city.apply(best_tmax, axis = 1, stations = stations)
city['density'] = city['population']/city['area']
# city

AttributeError: ("module 'numpy' has no attribute 'atan2'", 'occurred at index 0')

In [ ]:
plt.plot(city['best_tmax'],city['density'], 'b.')
plt.title('Temperature vs Population Density')
plt.xlabel('Avg Max Temperature (\u00b0C)')
plt.ylabel('Population Density (people/km\u00b2)')
# plt.show()

In [ ]:
plt.savefig(result)